In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os

In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/train_data.npy');
path_to_labels = path.join(root_folder,'data/train_labels.npy');
path_to_mean_std = path.join(root_folder,'data/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
num_classes = 2;
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 25;
logging_interval = 30
dataset_divider = 0.2
net = cnn.resnet18(num_classes = num_classes);
net.to(device)

ResNet(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv1d(64, 64, kernel_size

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
train_data, test_data = cnn.GenerateTrainAndTestDataset(torch_data, torch_labels, 0.2)


In [5]:
mean_std = torch.tensor(np.load(path_to_mean_std))
mean_train = mean_std[0].repeat(train_data.__len__());
std_train = mean_std[1].repeat(train_data.__len__());
mean_test = mean_std[0].repeat(test_data.__len__());
std_test = mean_std[1].repeat(test_data.__len__());
transform = transforms.Compose([transforms.Normalize(mean_train, std_train)])
train_data.data = transform(train_data.data)
transform = transforms.Compose([transforms.Normalize(mean_test, std_test)])
test_data.data = transform(test_data.data)

/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:206: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mean = torch.tensor(mean, dtype=torch.float32)
/home/vtpc/.local/lib/python3.5/site-packages/torchvision/transforms/functional.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  std = torch.tensor(std, dtype=torch.float32)


In [6]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)




In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/resnet18_num_classes_' + str(num_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, test_loader)

Train Epoch: 0 [0/9779124 (0%)]	Loss: 0.73374366760253906250
Train Epoch: 0 [450000/9779124 (5%)]	Loss: 0.12842872738838195801
Train Epoch: 0 [900000/9779124 (9%)]	Loss: 0.05345929786562919617
Train Epoch: 0 [1350000/9779124 (14%)]	Loss: 0.03604525327682495117
Train Epoch: 0 [1800000/9779124 (18%)]	Loss: 0.02673674374818801880
Train Epoch: 0 [2250000/9779124 (23%)]	Loss: 0.02166705578565597534
Train Epoch: 0 [2700000/9779124 (28%)]	Loss: 0.01849695481359958649
Train Epoch: 0 [3150000/9779124 (32%)]	Loss: 0.01567799411714076996
Train Epoch: 0 [3600000/9779124 (37%)]	Loss: 0.01337447296828031540
Train Epoch: 0 [4050000/9779124 (41%)]	Loss: 0.01277418434619903564
Train Epoch: 0 [4500000/9779124 (46%)]	Loss: 0.01189560722559690475
Train Epoch: 0 [4950000/9779124 (51%)]	Loss: 0.00953766051679849625
Train Epoch: 0 [5400000/9779124 (55%)]	Loss: 0.00996732525527477264
Train Epoch: 0 [5850000/9779124 (60%)]	Loss: 0.00831378065049648285
Train Epoch: 0 [6300000/9779124 (64%)]	Loss: 0.008758017793

Train Epoch: 5 [1800000/9779124 (18%)]	Loss: 0.00358804292045533657
Train Epoch: 5 [2250000/9779124 (23%)]	Loss: 0.00281647569499909878
Train Epoch: 5 [2700000/9779124 (28%)]	Loss: 0.00325984926894307137
Train Epoch: 5 [3150000/9779124 (32%)]	Loss: 0.00280463229864835739
Train Epoch: 5 [3600000/9779124 (37%)]	Loss: 0.00248227664269506931
Train Epoch: 5 [4050000/9779124 (41%)]	Loss: 0.00437260046601295471
Train Epoch: 5 [4500000/9779124 (46%)]	Loss: 0.00245663197711110115
Train Epoch: 5 [4950000/9779124 (51%)]	Loss: 0.00319519219920039177
Train Epoch: 5 [5400000/9779124 (55%)]	Loss: 0.00330837373621761799
Train Epoch: 5 [5850000/9779124 (60%)]	Loss: 0.00290918443351984024
Train Epoch: 5 [6300000/9779124 (64%)]	Loss: 0.00312013179063796997
Train Epoch: 5 [6750000/9779124 (69%)]	Loss: 0.00266095763072371483
Train Epoch: 5 [7200000/9779124 (74%)]	Loss: 0.00333855627104640007
Train Epoch: 5 [7650000/9779124 (78%)]	Loss: 0.00338022876530885696
Train Epoch: 5 [8100000/9779124 (83%)]	Loss: 0.0

Train Epoch: 10 [3600000/9779124 (37%)]	Loss: 0.00337953306734561920
Train Epoch: 10 [4050000/9779124 (41%)]	Loss: 0.00259485444985330105
Train Epoch: 10 [4500000/9779124 (46%)]	Loss: 0.00276994472369551659
Train Epoch: 10 [4950000/9779124 (51%)]	Loss: 0.00334231462329626083
Train Epoch: 10 [5400000/9779124 (55%)]	Loss: 0.00334264524281024933
Train Epoch: 10 [5850000/9779124 (60%)]	Loss: 0.00257545313797891140
Train Epoch: 10 [6300000/9779124 (64%)]	Loss: 0.00307149463333189487
Train Epoch: 10 [6750000/9779124 (69%)]	Loss: 0.00304905488155782223
Train Epoch: 10 [7200000/9779124 (74%)]	Loss: 0.00285675679333508015
Train Epoch: 10 [7650000/9779124 (78%)]	Loss: 0.00358184403739869595
Train Epoch: 10 [8100000/9779124 (83%)]	Loss: 0.00308342021889984608
Train Epoch: 10 [8550000/9779124 (87%)]	Loss: 0.00291951489634811878
Train Epoch: 10 [9000000/9779124 (92%)]	Loss: 0.00265214196406304836
Train Epoch: 10 [9450000/9779124 (97%)]	Loss: 0.00296106189489364624

Test set: Average loss: 0.0000001

Train Epoch: 15 [4950000/9779124 (51%)]	Loss: 0.00319759943522512913
Train Epoch: 15 [5400000/9779124 (55%)]	Loss: 0.00306681916117668152
Train Epoch: 15 [5850000/9779124 (60%)]	Loss: 0.00305593037046492100
Train Epoch: 15 [6300000/9779124 (64%)]	Loss: 0.00404287409037351608
Train Epoch: 15 [6750000/9779124 (69%)]	Loss: 0.00332311796955764294
Train Epoch: 15 [7200000/9779124 (74%)]	Loss: 0.00353440805338323116
Train Epoch: 15 [7650000/9779124 (78%)]	Loss: 0.00410094438120722771
Train Epoch: 15 [8100000/9779124 (83%)]	Loss: 0.00364884594455361366
Train Epoch: 15 [8550000/9779124 (87%)]	Loss: 0.00336155761033296585
Train Epoch: 15 [9000000/9779124 (92%)]	Loss: 0.00314430659636855125
Train Epoch: 15 [9450000/9779124 (97%)]	Loss: 0.00347466720268130302

Test set: Average loss: 0.00000016999561580633, Accuracy: 2443890/2444780 (99.96359590638012093677%)

Train Epoch: 16 [0/9779124 (0%)]	Loss: 0.00324998074211180210
Train Epoch: 16 [450000/9779124 (5%)]	Loss: 0.00290982262231409550
Train Epo

Train Epoch: 20 [6300000/9779124 (64%)]	Loss: 0.00389508367516100407
Train Epoch: 20 [6750000/9779124 (69%)]	Loss: 0.00296954181976616383
Train Epoch: 20 [7200000/9779124 (74%)]	Loss: 0.00324984802864491940
Train Epoch: 20 [7650000/9779124 (78%)]	Loss: 0.00292240106500685215
Train Epoch: 20 [8100000/9779124 (83%)]	Loss: 0.00280098826624453068
Train Epoch: 20 [8550000/9779124 (87%)]	Loss: 0.00379519024863839149
Train Epoch: 20 [9000000/9779124 (92%)]	Loss: 0.00277848239056766033
Train Epoch: 20 [9450000/9779124 (97%)]	Loss: 0.00326353404670953751

Test set: Average loss: 0.00000017139640817732, Accuracy: 2443915/2444780 (99.96461849327955917488%)

Train Epoch: 21 [0/9779124 (0%)]	Loss: 0.00352256814949214458
Train Epoch: 21 [450000/9779124 (5%)]	Loss: 0.00342595297843217850
Train Epoch: 21 [900000/9779124 (9%)]	Loss: 0.00329277780838310719
Train Epoch: 21 [1350000/9779124 (14%)]	Loss: 0.00318344426341354847
Train Epoch: 21 [1800000/9779124 (18%)]	Loss: 0.00299170776270329952
Train Epoch

In [8]:
cnn.Test(net, device, criterion, test_loader)


Test set: Average loss: 0.00000017160273557693, Accuracy: 2443918/2444780 (99.96474120370749005815%)

